原始数据做一个简单的处理，统一转为utf-8格式的csv。

原始数据航班动态数据单个文件太大，处理后将航班动态数据按月拆分成小文件，数据量太大，数据就不上传到git了。

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

from datetime import datetime

%matplotlib inline


## 天气

In [3]:
train_w = pd.read_csv('input/train/raw/weather-201505-201705.csv', low_memory=False)

In [4]:
display(train_w.head())

train_w.info()

,"﻿""城市""",天气,最低气温,最高气温,日期,Unnamed: 5
0,厦门,阴,21,26,2015-05-01,NaN
1,宁德,阵雨,19,25,2015-05-01,NaN
2,莆田,阵雨转阴,20,26,2015-05-01,NaN
3,泉州,阴,22,26,2015-05-01,NaN
4,漳州,阵雨转阴,22,28,2015-05-01,NaN


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828469 entries, 0 to 828468
Data columns (total 6 columns):
﻿"城市"         828469 non-null object
天气            828399 non-null object
最低气温          778400 non-null object
最高气温          678990 non-null object
日期            828469 non-null object
Unnamed: 5    0 non-null float64
dtypes: float64(1), object(5)
memory usage: 37.9+ MB


In [5]:
train_w.drop('Unnamed: 5', axis=1, inplace=True)

train_w.columns = ['City', 'Weather', 'MinTp', 'MaxTp', 'Date']

train_w.head()

,City,Weather,MinTp,MaxTp,Date
0,厦门,阴,21,26,2015-05-01
1,宁德,阵雨,19,25,2015-05-01
2,莆田,阵雨转阴,20,26,2015-05-01
3,泉州,阴,22,26,2015-05-01
4,漳州,阵雨转阴,22,28,2015-05-01


In [6]:
train_w.to_csv('input/train/weather.csv', index=False, encoding='utf-8')

## 动态数据

In [7]:
train_f = pd.read_csv('input/train/raw/fight-201505-201705.csv', encoding='gb2312', low_memory=False)

display(train_f.head())
train_f.info()

,出发机场,到达机场,航班编号,计划起飞时间,计划到达时间,实际起飞时间,实际到达时间,飞机编号,航班是否取消
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7518638 entries, 0 to 7518637
Data columns (total 9 columns):
出发机场      object
到达机场      object
航班编号      object
计划起飞时间    int64
计划到达时间    int64
实际起飞时间    float64
实际到达时间    float64
飞机编号      float64
航班是否取消    object
dtypes: float64(3), int64(2), object(4)
memory usage: 516.3+ MB


In [8]:
#Actual departure time
train_f.columns = ['Departure', 'Destination', 'FLTNo', 'PDepartureTime', 'PArrivalTime', 'ADepartureTime', 'AArrivalTime', 'Id', 'IsCancel']

train_f.head()

,Departure,Destination,FLTNo,PDepartureTime,PArrivalTime,ADepartureTime,AArrivalTime,Id,IsCancel
0,HGH,DLC,CZ6328,1453809600,1453817100,1.453813e+09,1.453819e+09,1.0,正常
1,SHA,XMN,FM9261,1452760800,1452767100,1.452763e+09,1.452768e+09,2.0,正常
2,CAN,WNZ,ZH9597,1453800900,1453807500,1.453802e+09,1.453807e+09,3.0,正常
3,SHA,ZUH,9C8819,1452120600,1452131100,1.452121e+09,1.452130e+09,4.0,正常
4,SHE,TAO,TZ185,1452399000,1452406800,1.452400e+09,1.452404e+09,5.0,正常


In [9]:
print('Max PDepartureTime:', datetime.fromtimestamp(train_f['PDepartureTime'].max()).strftime('%Y-%m-%d %H:%M:%S'))
print('Min PDepartureTime:', datetime.fromtimestamp(train_f['PDepartureTime'].min()).strftime('%Y-%m-%d %H:%M:%S'))

Max PDepartureTime: 2017-05-31 23:55:00
Min PDepartureTime: 2015-05-01 00:00:00


In [10]:
months = [
    '2015-05', '2015-06', '2015-07', '2015-08', '2015-09', '2015-10',
    '2015-11', '2015-12', '2016-01', '2016-02', '2016-03', '2016-04',
    '2016-05', '2016-06', '2016-07', '2016-08', '2016-09', '2016-10',
    '2016-11', '2016-12', '2017-01', '2017-02', '2017-03', '2017-04',
    '2017-05', '2017-06'
]

for i in range(len(months)):
    if i == len(months) - 1:
        continue

    start = datetime.strptime('%s-01' % months[i], '%Y-%m-%d').timestamp()
    end = datetime.strptime('%s-01' % months[i + 1], '%Y-%m-%d').timestamp()
    
    train_f_sub = train_f[(train_f['PDepartureTime'] >= start) & (train_f['PDepartureTime'] < end)]
    print(months[i], train_f_sub.size)
    train_f_sub.to_csv('input/train/fights-%s.csv' % months[i], index=False, encoding='utf-8')


2015-05 2482263
2015-06 2437416
2015-07 2723553
2015-08 2764080
2015-09 2595186
2015-10 2621691
2015-11 2426958
2015-12 2536389
2016-01 2636244
2016-02 2561193
2016-03 2619702
2016-04 2607273
2016-05 2644839
2016-06 2593386
2016-07 2892096
2016-08 2933685
2016-09 2758509
2016-10 2852757
2016-11 2731842
2016-12 2825874
2017-01 2982942
2017-02 2735865
2017-03 2911293
2017-04 2853693
2017-05 2939013


## 特情

In [12]:
train_s = pd.read_excel('input/train/raw/special-case-201505-201705.xlsx')

display(train_s.head())
train_s.info()

,特情机场,收集时间,开始时间,结束时间,特情内容
0,csx,2015-04-30 19:52:45Z,2015-05-01 12:00:00Z,2015-05-01 16:00:00Z,长沙机场航班延误黄色预警提示：预计明日（5月1日）12:00-16:00长沙黄花机场受雷雨天...
1,wuh,2015-05-01 08:13:39Z,2015-05-01 08:00:00Z,2015-05-01 12:00:00Z,目前，8时-12时雷雨，期间进出港航班可能受到影响。
2,khn,2015-05-01 08:14:18Z,2015-05-01 14:00:00Z,2015-05-01 17:00:00Z,预计，14时-17时小雷雨，期间进出港航班可能受到影响。
3,swa,2015-05-01 08:24:28Z,2015-05-01 13:00:00Z,2015-05-01 17:00:00Z,预计，13时-17时雷雨，期间进出港航班可能受到影响。
4,zha,2015-05-01 08:24:56Z,2015-05-01 13:00:00Z,2015-05-01 17:00:00Z,预计，13时-17时雷雨，期间进出港航班可能受到影响。


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15880 entries, 0 to 15879
Data columns (total 5 columns):
特情机场    15877 non-null object
收集时间    15869 non-null object
开始时间    15869 non-null object
结束时间    15862 non-null object
特情内容    15868 non-null object
dtypes: object(5)
memory usage: 620.4+ KB


In [13]:
train_s.columns = ['Airport', 'CTime', 'STime', 'ETime', 'Content']
train_s.head()

,Airport,CTime,STime,ETime,Content
0,csx,2015-04-30 19:52:45Z,2015-05-01 12:00:00Z,2015-05-01 16:00:00Z,长沙机场航班延误黄色预警提示：预计明日（5月1日）12:00-16:00长沙黄花机场受雷雨天...
1,wuh,2015-05-01 08:13:39Z,2015-05-01 08:00:00Z,2015-05-01 12:00:00Z,目前，8时-12时雷雨，期间进出港航班可能受到影响。
2,khn,2015-05-01 08:14:18Z,2015-05-01 14:00:00Z,2015-05-01 17:00:00Z,预计，14时-17时小雷雨，期间进出港航班可能受到影响。
3,swa,2015-05-01 08:24:28Z,2015-05-01 13:00:00Z,2015-05-01 17:00:00Z,预计，13时-17时雷雨，期间进出港航班可能受到影响。
4,zha,2015-05-01 08:24:56Z,2015-05-01 13:00:00Z,2015-05-01 17:00:00Z,预计，13时-17时雷雨，期间进出港航班可能受到影响。


In [14]:
train_s.to_csv('input/train/special-case.csv', index=False, encoding='utf-8')

## 机场城市对应

In [15]:
train_a = pd.read_excel('input/train/raw/airport-city.xlsx')
train_a.head()

,机场编码,城市名称
0,AHJ,阿坝
1,AYN,安阳
2,HSC,韶关
3,HCJ,河池
4,XNT,邢台


In [16]:
train_a.columns = ['Airport', 'City']
train_a.head()

,Airport,City
0,AHJ,阿坝
1,AYN,安阳
2,HSC,韶关
3,HCJ,河池
4,XNT,邢台


In [17]:
train_a.to_csv('input/train/airport-city.csv', index=False, encoding='utf-8')